In [29]:
from gc import collect
import logging
import time

import epics

from ophyd import (Device, Component as Cpt, FormattedComponent as FC,
                   Signal)
from ophyd import (EpicsSignal, EpicsSignalRO, DeviceStatus, DerivedSignal)
from ophyd.areadetector import EpicsSignalWithRBV as SignalWithRBV
from ophyd.status import SubscriptionStatus, Status
#from ophyd import status

In [57]:
class QEProTEC(Device):

    # Thermal electric cooler settings
    tec = Cpt(SignalWithRBV, 'TEC')
    tec_temp = Cpt(SignalWithRBV, 'TEC_TEMP', kind='config')
    curr_tec_temp = Cpt(EpicsSignalRO, 'CURR_TEC_TEMP_RBV')

    def __init__(self, *args, tolerance=1, **kwargs):
        self.tolerance = tolerance
        super().__init__(*args, **kwargs)

    def set(self, value):

        def check_setpoint(value, old_value, **kwargs):
            if abs(value - self.tec_temp.get()) < self.tolerance:
                print(f'Reached setpoint {self.tec_temp.get()}.')
                return True
            return False

        status = SubscriptionStatus(self.curr_tec_temp, run=False, callback=check_setpoint)
        self.tec_temp.put(value)
        self.tec.put(1)

        return status


class QEPro(Device):

    # Device information
    serial = Cpt(EpicsSignal, 'SERIAL')
    model = Cpt(EpicsSignal, 'MODEL')

    # Device Status
    status = Cpt(EpicsSignal, 'STATUS')
    status_msg = Cpt(EpicsSignal, 'STATUS_MSG')
    device_connected = Cpt(EpicsSignalRO, 'CONNECTED_RBV')

    # Utility signal that periodically checks device temps.
    __check_status = Cpt(EpicsSignal, 'CHECK_STATUS')

    # Bit array outlining which features are supported by the device
    features = Cpt(EpicsSignalRO, 'FEATURES_RBV')
    
    # Togglable features (if supported)
    strobe = Cpt(SignalWithRBV, 'STROBE')
    electric_dark_correction = Cpt(SignalWithRBV, 'EDC', kind='config')
    non_linearity_correction = Cpt(SignalWithRBV, 'NLC', kind='config')
    shutter = Cpt(SignalWithRBV, 'SHUTTER')
    
    # Thermal electric cooler
    tec_device = Cpt(QEProTEC, '')
    
    # Light source feature signals
    light_source = Cpt(SignalWithRBV, 'LIGHT_SOURCE')
    light_source_intensity = Cpt(SignalWithRBV, 'LIGHT_SOURCE_INTENSITY')
    light_source_count = Cpt(EpicsSignalRO, 'LIGHT_SOURCE_COUNT_RBV')

    # Signals for specifying the number of spectra to average and counter for spectra
    # collected in current scan
    num_spectra = Cpt(SignalWithRBV, 'NUM_SPECTRA', kind='hinted')
    spectra_collected = Cpt(EpicsSignalRO, 'SPECTRA_COLLECTED_RBV')

    # Integration time settings (in ms)
    int_min_time = Cpt(EpicsSignalRO, 'INT_MIN_TIME_RBV')
    int_max_time = Cpt(EpicsSignalRO, 'INT_MAX_TIME_RBV')
    integration_time = Cpt(SignalWithRBV, 'INTEGRATION_TIME', kind='hinted')
    
    # Internal buffer feature settings
    buff_min_capacity = Cpt(EpicsSignalRO, 'BUFF_MIN_CAPACITY_RBV')
    buff_max_capacity = Cpt(EpicsSignalRO, 'BUFF_MAX_CAPACITY_RBV')
    buff_capacity = Cpt(SignalWithRBV, 'BUFF_CAPACITY')
    buff_element_count = Cpt(EpicsSignalRO, 'BUFF_ELEMENT_COUNT_RBV')

    # Formatted Spectra
    output = Cpt(EpicsSignal, 'OUTPUT', kind='hinted')
    sample = Cpt(EpicsSignal, 'SAMPLE', kind='hinted')
    dark = Cpt(EpicsSignal, 'DARK', kind='hinted')
    reference = Cpt(EpicsSignal, 'REFERENCE', kind='hinted')
    
    # Length of spectrum (in pixels)
    formatted_spectrum_len = Cpt(EpicsSignalRO, 'FORMATTED_SPECTRUM_LEN_RBV')

    # X-axis format and array
    x_axis = Cpt(EpicsSignal, 'X_AXIS', kind='hinted')
    x_axis_format = Cpt(SignalWithRBV, 'X_AXIS_FORMAT')

    # Dark/Ref available signals
    dark_available = Cpt(EpicsSignalRO, 'DARK_AVAILABLE_RBV')
    ref_available = Cpt(EpicsSignalRO, 'REF_AVAILABLE_RBV')

    # Collection settings and start signals.
    acquire = Cpt(SignalWithRBV, 'COLLECT')
    collect_mode = Cpt(SignalWithRBV, 'COLLECT_MODE', kind='hinted')
    spectrum_type = Cpt(SignalWithRBV, 'SPECTRUM_TYPE', kind='hinted')
    correction = Cpt(SignalWithRBV, 'CORRECTION', kind='hinted')
    trigger_mode = Cpt(SignalWithRBV, 'TRIGGER_MODE')


    @property
    def has_nlc_feature(self):
        return self.features.get() & 32

    @property
    def has_lightsource_feature(self):
        return self.features.get() & 16

    @property
    def has_edc_feature(self):
        return self.features.get() & 8

    @property
    def has_buffer_feature(self):
        return self.features.get() & 4

    @property
    def has_tec_feature(self):
        return self.features.get() & 2

    @property
    def has_irrad_feature(self):
        return self.features.get() & 1


    def set_temp(self, temperature):
        self.tec_device.set(temperature).wait()


    def get_dark_frame(self):

        current_spectrum = self.spectrum_type.get()
        self.spectrum_type.put('Dark')
        self.acquire.put(1, wait=True)
        time.sleep(1)
        self.spectrum_type.put(current_spectrum)
    
    def get_reference_frame(self):

        current_spectrum = self.spectrum_type.get()
        self.spectrum_type.put('Reference')
        self.acquire.put(1, wait=True)
        time.sleep(1)
        self.spectrum_type.put(current_spectrum)


    def setup_collection(self, integration_time=100, num_spectra_to_average=10, spectrum_type='Absorbtion', correction_type='Reference', electric_dark_correction=True):
        self.integration_time.put(integration_time)
        self.num_spectra.put(num_spectra_to_average)
        if num_spectra_to_average > 1:
            self.collect_mode.put('Average')
        else:
            self.collect_mode.put('Single')

        if electric_dark_correction:
            self.electric_dark_correction.put(1)

        self.correction.put(correction_type)

        self.spectrum_type.put(spectrum_type)


    def grab_frame(self):

        def is_done(value, old_value, **kwargs):
            if old_value == 1 and value ==0:
                return True
            return False

        status = SubscriptionStatus(self.acquire, run=False, callback=is_done)

        self.acquire.put(1)
        return status

    def trigger(self):
        #self.grab_frame().wait()
        return self.grab_frame()

    def write_as_csv(self, write_path):
    
        print(f'Writing out CSV file to {write_path}...')

        with open(write_path, 'w') as fp:
            x_axis_data = self.x_axis.get()
            output_data = self.output.get()
            sample_data = self.sample.get()
            dark_data = self.dark.get()
            reference_data = self.reference.get()
            if self.spectrum_type.get(as_string=True) == 'Absorbtion':
                fp.write('Energy,Dark,Reference,Sample,Absorbtion\n')
            else:
                fp.write('Energy,Dark,Raw Sample,Corrected Sample\n')

            for i in range(len(output_data)):
                if self.spectrum_type.get(as_string=True) == 'Absorbtion':
                    fp.write(f'{x_axis_data[i]},{dark_data[i]},{reference_data[i]},{sample_data[i]},{output_data[i]}\n')
                else:
                    fp.write(f'{x_axis_data[i]},{dark_data[i]},{sample_data[i]},{output_data[i]}\n')

            print('Done.')

    def plot_spectra(self):
        x_axis_data = self.x_axis.get()
        output_data = self.output.get()

        x_axis_label = self.x_axis_format.get(as_string=True)
        y_axis_label = self.spectrum_type.get(as_string=True)


        plt.plot(x_axis_data, output_data)
        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.show()


qepro = QEPro('XF:28ID2-ES{QEPro:Spec-1}:', name='QEPro', )


In [18]:
qepro.set_temp(-5)

Reached setpoint -5.0.


In [31]:
qepro.get_dark_frame()

In [32]:
qepro.get_reference_frame()

In [33]:
qepro.setup_collection()

In [34]:
RE(count([qepro]))



Transient Scan ID: 3     Time: 2022-10-21 14:51:31
Persistent Unique Scan ID: '36c25898-c47c-4444-ac08-4e34bdf05564'
QEPro_acquire [In progress. No progress bar available.]                         
QEPro_acquire [In progress. No progress bar available.]                         
                                                                                
New stream: 'primary'


/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/callbacks/core.py:332: UserWarning: The key QEPro_output will be skipped because LiveTable does not know how to display the dtype array
  warnings.warn("The key {} will be skipped because LiveTable "
/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/callbacks/core.py:332: UserWarning: The key QEPro_sample will be skipped because LiveTable does not know how to display the dtype array
  warnings.warn("The key {} will be skipped because LiveTable "
/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/callbacks/core.py:332: UserWarning: The key QEPro_dark will be skipped because LiveTable does not know how to display the dtype array
  warnings.warn("The key {} will be skipped because LiveTable "
/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/callbacks/core.py:332: UserWarning: The key QEPro_reference will be skipped because LiveTable does not kno

+-----------+------------+-------------------+------------------------+--------------------+---------------------+------------------+
|   seq_num |       time | QEPro_num_spectra | QEPro_integration_time | QEPro_collect_mode | QEPro_spectrum_type | QEPro_correction |
+-----------+------------+-------------------+------------------------+--------------------+---------------------+------------------+
|         1 | 14:51:33.1 |                10 |                100.000 |                  1 |                   3 |                2 |
+-----------+------------+-------------------+------------------------+--------------------+---------------------+------------------+
generator count ['36c25898'] (scan num: 3)





('36c25898-c47c-4444-ac08-4e34bdf05564',)

In [50]:
db[-1].table().columns

Index(['QEPro_serial', 'QEPro_model', 'QEPro_status', 'QEPro_status_msg',
       'QEPro_device_connected', 'QEPro__QEPro__check_status',
       'QEPro_features', 'QEPro_strobe', 'QEPro_shutter',
       'QEPro_tec_device_tec', 'QEPro_tec_device_curr_tec_temp',
       'QEPro_light_source', 'QEPro_light_source_intensity',
       'QEPro_light_source_count', 'QEPro_num_spectra',
       'QEPro_spectra_collected', 'QEPro_int_min_time', 'QEPro_int_max_time',
       'QEPro_integration_time', 'QEPro_buff_min_capacity',
       'QEPro_buff_max_capacity', 'QEPro_buff_capacity',
       'QEPro_buff_element_count', 'QEPro_output', 'QEPro_sample',
       'QEPro_dark', 'QEPro_reference', 'QEPro_formatted_spectrum_len',
       'QEPro_x_axis', 'QEPro_x_axis_format', 'QEPro_dark_available',
       'QEPro_ref_available', 'QEPro_acquire', 'QEPro_collect_mode',
       'QEPro_spectrum_type', 'QEPro_correction', 'QEPro_trigger_mode',
       'time'],
      dtype='object')

In [47]:
qepro.dark.get()

array([10.875, -2.125, -1.125, ...,  0.875, -4.125, -0.125])

In [56]:
db[-1].table().QEPro_dark

seq_num
1    [10.875, -2.125, -1.125, -2.125, -2.125, -2.12...
Name: QEPro_dark, dtype: object